In [ ]:
import orca
import urbansim

import datasources
import variables

In [ ]:
j = orca.get_table('jobs').to_frame()
z = orca.get_table('zones').to_frame()

In [ ]:
z.columns

In [ ]:
expvars = ['total_household_population', 'travel_time_to_cbd_drive', 'avg_year_built', 'total_number_of_jobs']
dcm = urbansim.models.dcm.MNLDiscreteChoiceModel(model_expression=expvars,
                                                 sample_size=50,
                                                 probability_mode='single_chooser',
                                                 choice_mode='aggregate',
                                                 estimation_sample_size=1000)

In [ ]:
dcm.fit(j, z, 'zone_id')

In [1]:
import numpy as np
import pandas as pd
from urbansim.models import MNLDiscreteChoiceModel
from urbansim.utils import misc


from sklearn.utils.validation import check_X_y
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.estimator_checks import check_estimator

from sklearn.metrics import euclidean_distances

from sklearn.base import BaseEstimator, ClassifierMixin

class MnlClassifier(MNLDiscreteChoiceModel, BaseEstimator, ClassifierMixin):

    def __init__(self, model_expression='', sample_size=100):
         super(MnlClassifier, self).__init__(model_expression, sample_size)
        
    def fit(self, X, y):
        ##  X here should be MNLDiscreteChoiceModel.fit, with pre-merged choosers/alts...
        # Check that X and y have correct shape
        X, y = check_X_y(X, y)
        # Store the classes seen during fit
        self.classes_ = unique_labels(y)

        self.X_ = X
        self.y_ = y
        # Return the classifier
        return self
    
    def predict(self, X):

        # Check is fit had been called
        check_is_fitted(self, ['X_', 'y_'])

        # Input validation
        X = check_array(X)

        closest = np.argmin(euclidean_distances(X, self.X_), axis=1)
        return self.y_[closest]
    
check_estimator(MnlClassifier)

NameError: global name 'unique_labels' is not defined

In [2]:
from bokeh.plotting import figure, output_notebook, show
output_notebook()

from datashader import transfer_functions as tf
from datashader.colors import Greys9
Greys9_r = list(reversed(Greys9))[:-2]

from bokeh.tile_providers import STAMEN_TERRAIN

import datashader as ds
from datashader.bokeh_ext import InteractiveImage
from functools import partial
from datashader.utils import export_image
from datashader.colors import colormap_select, Greys9, Hot, viridis, inferno
from IPython.core.display import HTML, display

def plot_agents(df, name, sum_attrib=None, theme=Hot):
    df['ones'] = 1
    region = x_range, y_range = ((df.x.min(),df.x.max()), (df.y.min(),df.y.max()))

    plot_width  = int(750)
    plot_height = int(plot_width//1.2)

    def base_plot(tools='pan,wheel_zoom,reset',plot_width=plot_width, plot_height=plot_height, **plot_args):
        p = figure(tools=tools, plot_width=plot_width, plot_height=plot_height,
            x_range=x_range, y_range=y_range, outline_line_color=None,
            min_border=0, min_border_left=0, min_border_right=0,
            min_border_top=0, min_border_bottom=0, **plot_args)

        p.axis.visible = False
        p.xgrid.grid_line_color = None
        p.ygrid.grid_line_color = None
        return p

    options = dict(line_color=None, fill_color='blue', size=5)

    background = "black"
    export = partial(export_image, export_path="export", background=background)
    cm = partial(colormap_select, reverse=(background=="black"))

    def create_image(x_range, y_range, w=plot_width, h=plot_height):
        cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
        if sum_attrib:
            agg = cvs.points(df, 'x', 'y',  ds.count(sum_attrib))
        else:
            agg = cvs.points(df, 'x', 'y',  ds.count('ones'))
        img = tf.shade(agg, cmap=theme, how='eq_hist')
        return tf.dynspread(img, threshold=0.5, max_px=4)

    p = base_plot(background_fill_color=background)
    #p.add_tile(STAMEN_TERRAIN)
    export(create_image(*region), name)
    return InteractiveImage(p, create_image)

def plot_agents_by_cat(df, name):
    df['ones'] = 1
    region = x_range, y_range = ((df.x.min(),df.x.max()), (df.y.min(),df.y.max()))

    plot_width  = int(750)
    plot_height = int(plot_width//1.2)

    def base_plot(tools='pan,wheel_zoom,reset',plot_width=plot_width, plot_height=plot_height, **plot_args):
        p = figure(tools=tools, plot_width=plot_width, plot_height=plot_height,
            x_range=x_range, y_range=y_range, outline_line_color=None,
            min_border=0, min_border_left=0, min_border_right=0,
            min_border_top=0, min_border_bottom=0, **plot_args)

        p.axis.visible = False
        p.xgrid.grid_line_color = None
        p.ygrid.grid_line_color = None
        return p

    options = dict(line_color=None, fill_color='blue', size=5)

    background = "black"
    export = partial(export_image, export_path="export", background=background)
    cm = partial(colormap_select, reverse=(background=="black"))

    def create_image(x_range, y_range, w=plot_width, h=plot_height):
        cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
        color_key = color_key = {'low':'aqua', 'high':'red'}

        agg = cvs.points(df, 'x', 'y',  ds.count_cat('proba_class'))

        img = tf.shade(agg, color_key=color_key, how='eq_hist')
        return tf.dynspread(img, threshold=0.5, max_px=4)

    p = base_plot(background_fill_color=background)
    #p.add_tile(STAMEN_TERRAIN)
    export(create_image(*region), name)
    return InteractiveImage(p, create_image)

Loading BokehJS ...